## Importación de librerías

In [2]:
import cv2
import dlib
import cv2
from random import randint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Creación de tracker

In [3]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

In [4]:
def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
  if trackerType == trackerTypes[0]:
    tracker = cv2.TrackerBoosting_create()
  elif trackerType == trackerTypes[1]:
    tracker = cv2.TrackerMIL_create()
  elif trackerType == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType == trackerTypes[3]:
    tracker = cv2.TrackerTLD_create()
  elif trackerType == trackerTypes[4]:
    tracker = cv2.TrackerMedianFlow_create()
  elif trackerType == trackerTypes[5]:
    tracker = cv2.TrackerGOTURN_create()
  elif trackerType == trackerTypes[6]:
    tracker = cv2.TrackerMOSSE_create()
  elif trackerType == trackerTypes[7]:
    tracker = cv2.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)

  return tracker

## Inicialización de vídeo

In [6]:
cap = cv2.VideoCapture(0)

if(cap.isOpened()):
    print('WebCam succesfully opened')

WebCam succesfully opened


## Bucle principal de detección + tracking de facial

In [8]:
# Face detector
face_detector = dlib.get_frontal_face_detector()

# Obtener primer frame
if(cap.isOpened()):
    ret,frame = cap.read()

    h,w,c = frame.shape

    nframe = 0
    noTracking = False
    ok = True

    while(1):
        timer = cv2.getTickCount()
        ret, frame = cap.read()
        frame = cv2.resize(frame,(int(w/2),int(h/2)))
        frame_rect = frame.copy()

        if(nframe == 0):
            # Face detection
            detector_status = 'Detection'
            noTracking = False
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_detector(gray)
            color = (255,0,0)

            #Initializa Multi tracker variables
            bboxes = []
            colors = []

            # Create Multi tracker
            multiTracker = cv2.MultiTracker_create()

            for face in faces:
                # Get the bbox
                x1 = face.left()
                y1 = face.top()
                x2 = face.right()
                y2 = face.bottom()
                bbox = (x1,y1,x2-x1,y2-y1)
                bboxes.append(bbox)

                # Create random color
                color = (randint(64, 255), randint(64, 255),randint(64, 255))
                colors.append(color)

                # Create new tracker
                tracker = createTrackerByName(trackerTypes[2])
                multiTracker.add(tracker, frame, bbox)

                frame_rect = cv2.rectangle(frame_rect,(x1,y1),(x2,y2), color = color, thickness = 3)
            if(len(faces)>0):
                nframe+=1
        else:
            # Face tracking
            detector_status = 'Tracking'

            # Actualizamos el tracking
            ok, bboxs = multiTracker.update(frame)
            if(ok):
                nframe+=1
                for i,bbox in enumerate(bboxs):
                    p1 = (int(bbox[0]), int(bbox[1]))
                    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                    frame_rect = cv2.rectangle(frame_rect,p1,p2, color = colors[i], thickness = 3)

            else:
                noTracking = True
                detector_status = 'Tracking lost'

        if(nframe == 50 or noTracking):
            # Cada x frames o si falla el tracking pasamos a detección
            nframe = 0

        if(cv2.waitKey(0) & 0xFF == ord('q')):
            break

        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);

        cv2.putText(frame_rect,'FPS: {} Status: {}'.format(int(fps),detector_status),(20,40), cv2.FONT_HERSHEY_SIMPLEX,1, color = color)
        cv2.imshow('Image', frame_rect)


    cap.release()
    cv2.destroyAllWindows()
else:
    print('No WebCam has been initialized.')

No WebCam has been initialized.
